### Preparación del ambiente

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from scipy.stats import poisson
import numpy as np

In [3]:
pwd

'/content'

In [4]:
cd /content/drive/MyDrive/'Colab Notebooks'/laliga-nb

/content/drive/.shortcut-targets-by-id/1Wx_iFi2WCSDzrJxFdNMiae9Ks9HPSpMf/laliga-nb


## Lectura de los datos

In [26]:
df_historical_results = pd.read_csv('./historical_results.csv')
df_current_results = pd.read_csv('./current_results.csv')

In [27]:
df_historical_results.shape


(1900, 5)

In [28]:
df_current_results.shape

(260, 5)

In [29]:
df_total = pd.concat([df_historical_results,df_current_results])
df_total

,Local,Visitante,Temporada,GolesLocal,GolesVisitante,Fecha
0,Alavés,Athletic Bilbao,2017-18,3,1,NaN
1,Alavés,Atlético Madrid,2017-18,0,1,NaN
2,Alavés,Barcelona,2017-18,0,2,NaN
3,Alavés,Celta Vigo,2017-18,2,1,NaN
4,Alavés,Deportivo La Coruña,2017-18,1,0,NaN
...,...,...,...,...,...,...
255,Real Betis,Mallorca,NaN,1,0,2023-03-19
256,Osasuna,Villarreal,NaN,0,3,2023-03-19
257,Real Sociedad,Elche,NaN,2,0,2023-03-19
258,Getafe,Sevilla,NaN,2,0,2023-03-19


In [30]:
df_total = df_total.drop(['Temporada', 'Fecha'], axis=1)

In [32]:
df_total

,Local,Visitante,GolesLocal,GolesVisitante
0,Alavés,Athletic Bilbao,3,1
1,Alavés,Atlético Madrid,0,1
2,Alavés,Barcelona,0,2
3,Alavés,Celta Vigo,2,1
4,Alavés,Deportivo La Coruña,1,0
...,...,...,...,...
255,Real Betis,Mallorca,1,0
256,Osasuna,Villarreal,0,3
257,Real Sociedad,Elche,2,0
258,Getafe,Sevilla,2,0


##***PROBABILIDAD FÓRMULA POISSON***


In [11]:
# Crear una función para calcular la fuerza goleadora de los equipos como locales o visitantes
def team_strength(df_total):

    # Creando nuevos df de local y visitante
    df_home = df_total[['Local', 'GolesLocal', 'GolesVisitante']]
    df_away = df_total[['Visitante', 'GolesVisitante', 'GolesLocal']]

    # Dividiendo df en local y visitante
    df_home = df_home.rename(columns={'Local':'Equipo', 'GolesLocal': 'GolesAnotados', 'GolesVisitante': 'GolesRecibidos'})
    df_away = df_away.rename(columns={'Visitante':'Equipo', 'GolesVisitante': 'GolesAnotados', 'GolesLocal': 'GolesRecibidos'})

    # Calculando promedios totales de goles anotados por locales y visitantes
    home_scored = df_home['GolesAnotados'].mean() # goles anotados local = gples encajados visitante
    away_scored = df_away['GolesAnotados'].mean() # goles anotados visitante = gples encajados local

    # Calculando promedios de goles anotados y encajados por equipo local, en función del promedio total de goles anotados y encajados de local por todos los equipos
    df_homestrength = df_home.groupby(['Equipo']).mean()
    df_homestrength['PromedioGolesAnotados'] = df_homestrength['GolesAnotados'] / home_scored
    df_homestrength['PromedioGolesRecibidos'] = df_homestrength['GolesRecibidos'] / away_scored
    df_homestrength.drop(['GolesAnotados', 'GolesRecibidos'], axis=1, inplace=True) #Eliminar las filas que no necesitamos

    # Calculando promedios de goles anotados y encajados por equipo visitante en función del promedio total de goles anotados y encajados de visitante
    df_awaystrength = df_away.groupby(['Equipo']).mean()
    df_awaystrength['PromedioGolesAnotados'] = df_awaystrength['GolesAnotados'] / away_scored
    df_awaystrength['PromedioGolesRecibidos'] = df_awaystrength['GolesRecibidos'] / home_scored
    df_awaystrength.drop(['GolesAnotados', 'GolesRecibidos'], axis=1, inplace=True) #Eliminar las filas que no necesitamos

    # Imprimiendo dataframes
    return (df_homestrength, df_awaystrength, home_scored, away_scored)

#Llamando a la función team_strength
df_homestrength, df_awaystrength, home_scored, away_scored = team_strength(df_total) 

#Calcular la probabilidad de Poisson
def prob_poisson(home, away, df_homestrength, df_awaystrength, home_scored, away_scored):

    #Definiendo la función
    if home in df_homestrength.index and away in df_awaystrength.index:
        # goals_scored * goals_conceded
        home_xgoals = df_homestrength.at[home,'PromedioGolesAnotados'] * df_awaystrength.at[away, 'PromedioGolesRecibidos'] * home_scored
        away_xgoals = df_awaystrength.at[away,'PromedioGolesAnotados'] * df_homestrength.at[home,'PromedioGolesRecibidos'] * away_scored
        #return (goles_local, goles_visitante)
        prob_draw = 0
        prob_home = 0
        prob_away = 0

        for x in range(0,10): #Goles equipo local
            for y in range(0, 10): #Goles equipo visitante
                p = poisson.pmf(x, home_xgoals) * poisson.pmf(y, away_xgoals)
                if x == y:
                    prob_draw += p * 100
                elif x > y:
                    prob_home += p * 100
                else:
                    prob_away += p * 100

    # Imprimiendo probabilidades           
    return (f"{prob_home:.2f}%", f"{prob_draw:.2f}%", f"{prob_away:.2f}%")



In [13]:
#Probando la función
prob_poisson('Almería', 'Real Madrid', df_homestrength, df_awaystrength, home_scored, away_scored)

('18.99%', '20.54%', '60.47%')

## ***PROBABILIDAD ENFRENTAMIENTO DIRECTO***

In [14]:
def prob_headtohead(home, away, df_total):
      if ((df_total['Local'] == home).any() and (df_total['Local'] == away).any()):
        #sacar todos los partidos previos entre equipos
        matches1 = df_total.loc[(df_total['Local'] == home) & (df_total['Visitante'] == away)]
        matches2 = df_total.loc[(df_total['Local'] == away) & (df_total['Visitante'] == home)]

        # Creamos una copia del DataFrame
        home_results = matches1.copy()
        away_results = matches2.copy()

        #Cuando el equipo local juega de local / visitante de visitante:
        home_results.loc[home_results["GolesLocal"] > home_results["GolesVisitante"], "ResultadoLocal"] = "Local"
        home_results.loc[home_results["GolesLocal"] == home_results["GolesVisitante"], "ResultadoLocal"] = "Empate"
        home_results.loc[home_results["GolesLocal"] < home_results["GolesVisitante"], "ResultadoLocal"] = "Visitante"

        #Cuando el equipo local juega de visitante / visitante de local:
        away_results.loc[away_results["GolesVisitante"] > away_results["GolesLocal"], "ResultadoVisitante"] = "Visitante"
        away_results.loc[away_results["GolesVisitante"] == away_results["GolesLocal"], "ResultadoVisitante"] = "Empate"
        away_results.loc[away_results["GolesVisitante"] < away_results["GolesLocal"], "ResultadoVisitante"] = "Local"

        # contar el número de partidos por resultado
        home_results1 = home_results.groupby(["Local", "ResultadoLocal"]).size() * 2    #El partido especificado se juega con estas condiciones, por eso vale doble
        away_results1 = away_results.groupby(["Visitante", "ResultadoVisitante"]).size()

        # Calcular las probabilidades de victoria, empate y derrota de cada equipo
        prob_home = ((home_results1.get((home, "Local"), 0) + away_results1.get((home, "Visitante"), 0)) / (len(home_results) * 2 + len(away_results))) * 100
        prob_draw = ((home_results1.get((home, "Empate"), 0) + away_results1.get((home, "Empate"), 0)) / (len(home_results) * 2 + len(away_results))) * 100
        prob_away = ((home_results1.get((home, "Visitante"), 0) + away_results1.get((home, "Local"), 0)) / (len(home_results) * 2 + len(away_results))) * 100

        return(f"{prob_home:.2f}%", f"{prob_draw:.2f}%", f"{prob_away:.2f}%")



In [16]:
# Probando la función
prob_headtohead('Almería', 'Real Madrid', df_total)

ValueError: ignored

## ***PROBABILIDAD FORMA RECIENTE***

In [17]:
def last_matches_team(team, matches_num, df_current_results):

    #Filtrar los partidos del equipo especificado
    last_matches = df_current_results[(df_current_results['Local'] == team)| (df_current_results['Visitante'] == team)]
    
    #Ordenar los partidos por fecha descendente
    last_matches = last_matches.sort_values(by='Fecha', ascending=False)
    
    #Seleccionar los últimos (matches_num) partidos
    last_matches = last_matches.head(matches_num).reset_index(drop=True)

    #Asignar un peso a cada partido
    weights = np.exp(np.linspace(0, -1, matches_num))

    #Crear un nuevo DataFrame solo con los pesos
    weights_df = pd.DataFrame(weights, columns=['Pesos'])
    
    #Unir el nuevo DataFrame de los pesos con el DataFrame de los partidos locales y visitantes
    last_matches = pd.concat([last_matches, weights_df], axis=1)

    #Devolver el DataFrame con los últimos partidos del equipo
    return(last_matches)



In [19]:
# Probando la función
last_matches_team('Almería', 15, df_current_results)

,Local,Visitante,Fecha,GolesLocal,GolesVisitante,Pesos
0,Almería,Cádiz,2023-03-18,1,1,1.000000
1,Sevilla,Almería,2023-03-12,2,1,0.931063
2,Almería,Villarreal,2023-03-04,0,2,0.866878
3,Almería,Barcelona,2023-02-26,1,0,0.807118
4,Girona,Almería,2023-02-17,6,2,0.751477
5,Almería,Real Betis,2023-02-11,2,3,0.699673
6,Rayo Vallecano,Almería,2023-02-06,2,0,0.651439
7,Almería,Espanyol,2023-01-27,3,1,0.606531
8,Valencia,Almería,2023-01-23,2,2,0.564718
9,Almería,Atlético Madrid,2023-01-15,1,1,0.525788


In [20]:
def prob_recentform(home, away, matches_num, df_current_results):
    if ((df_current_results['Local'] == home).any() and (df_current_results['Local'] == away).any()):

        #Obtener los últimos partidos de cada equipo
        home_matches = last_matches_team(home, matches_num, df_current_results)
        away_matches = last_matches_team(away, matches_num, df_current_results)

        #Cuando el equipo local juega de local:
        home1 = home_matches.loc[home_matches['Local'] == home]
        home_matches1 = home1.copy()
        home_matches1.loc[home_matches1["GolesLocal"] > home_matches1["GolesVisitante"], "Resultado"] = "Local"
        home_matches1.loc[home_matches1["GolesLocal"] == home_matches1["GolesVisitante"], "Resultado"] = "Empate"
        home_matches1.loc[home_matches1["GolesLocal"] < home_matches1["GolesVisitante"], "Resultado"] = "Visitante"
        home_matches1['Pesos'] = home_matches1['Pesos'] * 2
        home_results1 = home_matches1.groupby('Resultado')['Pesos'].sum()

        #Cuando el equipo local juega de visitante:
        home2 = home_matches.loc[home_matches['Visitante'] == home]
        home_matches2 = home2.copy()
        home_matches2.loc[home_matches2["GolesLocal"] > home_matches2["GolesVisitante"], "Resultado"] = "Local"
        home_matches2.loc[home_matches2["GolesLocal"] == home_matches2["GolesVisitante"], "Resultado"] = "Empate"
        home_matches2.loc[home_matches2["GolesLocal"] < home_matches2["GolesVisitante"], "Resultado"] = "Visitante"
        home_matches2['Pesos'] = home_matches2['Pesos']
        home_results2 = home_matches2.groupby('Resultado')['Pesos'].sum()

        #Cuando el equipo visitante juega de visitante:
        away1 = away_matches.loc[away_matches['Visitante'] == away]
        away_matches1 = away1.copy()
        away_matches1.loc[away_matches1["GolesLocal"] > away_matches1["GolesVisitante"], "Resultado"] = "Local"
        away_matches1.loc[away_matches1["GolesLocal"] == away_matches1["GolesVisitante"], "Resultado"] = "Empate"
        away_matches1.loc[away_matches1["GolesLocal"] < away_matches1["GolesVisitante"], "Resultado"] = "Visitante"
        away_matches1['Pesos'] = away_matches1['Pesos'] * 2
        away_results1 = away_matches1.groupby('Resultado')['Pesos'].sum()

        #Cuando el equipo visitante juega de local:
        away2 = away_matches.loc[away_matches['Local'] == away]
        away_matches2 = away2.copy()
        away_matches2.loc[away_matches2["GolesLocal"] > away_matches2["GolesVisitante"], "Resultado"] = "Local"
        away_matches2.loc[away_matches2["GolesLocal"] == away_matches2["GolesVisitante"], "Resultado"] = "Empate"
        away_matches2.loc[away_matches2["GolesLocal"] < away_matches2["GolesVisitante"], "Resultado"] = "Visitante"
        away_matches2['Pesos'] = away_matches2['Pesos']
        away_results2 = away_matches2.groupby('Resultado')['Pesos'].sum()

        home_prob = home_results1.get('Local', 0) + home_results2.get('Visitante', 0) + away_results1.get('Local', 0) + away_results2.get('Visitante', 0)
        draw_prob = home_results1.get('Empate', 0) + home_results2.get('Empate', 0) + away_results1.get('Empate', 0) + away_results2.get('Empate', 0)
        away_prob = home_results1.get('Visitante', 0) + home_results2.get('Local', 0) + away_results1.get('Visitante', 0) + away_results2.get('Local', 0)
        
        total_prob = home_prob + draw_prob + away_prob

        prob_home = (home_prob / total_prob) * 100
        prob_draw = (draw_prob / total_prob) * 100
        prob_away = (away_prob / total_prob) * 100

        return(f"{prob_home:.2f}%", f"{prob_draw:.2f}%", f"{prob_away:.2f}%")




In [22]:
# Probando la función
prob_recentform('Real Madrid', 'Almería', 15, df_current_results)

('53.52%', '29.06%', '17.42%')